In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36'}

In [3]:
current_page_url = "https://www.bayut.om/en/oman/properties-for-rent/"
bas_url = "https://www.bayut.om/en"

In [4]:
response = requests.get(bas_url, headers=headers)
response.raise_for_status()
response

<Response [200]>

In [5]:
response_1 = requests.get(current_page_url, headers=headers)
response_1.raise_for_status()

In [6]:
soup1 = BeautifulSoup(response_1.text, 'html.parser')

In [ ]:
import time
import requests.compat
all_apt = []

while True:
    print("scarpping page: ", current_page_url)
    try:
        new_response = requests.get(current_page_url, headers=headers, timeout=10)
        new_response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"error fetching page {current_page_url}: {e}")
        break
    new_soup = BeautifulSoup(new_response.text, 'html.parser')
    apartments = new_soup.select('article')

    if not apartments:
        print("no apartment found!")
        break
    for i in apartments:
        n = i.find('div',class_='dde89f38').a.attrs['href']
        u1 = requests.compat.urljoin("https://www.bayut.om/en", n)
        #print(u1)
        r = requests.get(u1, headers=headers)
        s = BeautifulSoup(r.text, 'html.parser')
        d = s.select('div.a5a066fe')
        for j in d:
            title = j.find('h1').text
            location = j.find('div', class_='e4fd45f0').text
            price = j.find('div', class_='_2923a568').text
            size = j.find_all('span', class_="_140e6903")[-1].text
            listing_type = j.find_all('span', class_="_2fdf7fc5")[1].text
            # print(f"title: {title}")
            # print(f"location: {location}")
            # print(f"price: {price}")
            # print(f"size: {size}")
            # print(f"listing_type: {listing_type}")
            d1 = {'Title':title, 'Location':location, 'Price': price, 'Size': size, 'Listing_Type':listing_type}
            all_apt.append(d1)

    div_1 = new_soup.select('div.e27bf381')
    for link in div_1:
        try:
            next_button = link.find('ul').find_all('li')[-1].find('a').attrs['title']
            new_link = link.find('ul').find_all('li')[-1].find('a').attrs['href']
        except (AttributeError, IndexError, TypeError) as e:
            print("Error extracting pagination link:", e)
            next_button = ''
    if next_button:
        current_page_url = requests.compat.urljoin(current_page_url, new_link)
        time.sleep(1)
    else:
        print("no more apartment found!")
        break

scarpping page:  https://www.bayut.om/en/oman/properties-for-rent/
scarpping page:  https://www.bayut.om/en/oman/properties-for-rent/page-2/
scarpping page:  https://www.bayut.om/en/oman/properties-for-rent/page-3/
scarpping page:  https://www.bayut.om/en/oman/properties-for-rent/page-4/
scarpping page:  https://www.bayut.om/en/oman/properties-for-rent/page-5/
scarpping page:  https://www.bayut.om/en/oman/properties-for-rent/page-6/
scarpping page:  https://www.bayut.om/en/oman/properties-for-rent/page-7/
scarpping page:  https://www.bayut.om/en/oman/properties-for-rent/page-8/
scarpping page:  https://www.bayut.om/en/oman/properties-for-rent/page-9/
scarpping page:  https://www.bayut.om/en/oman/properties-for-rent/page-10/
scarpping page:  https://www.bayut.om/en/oman/properties-for-rent/page-11/
scarpping page:  https://www.bayut.om/en/oman/properties-for-rent/page-12/
scarpping page:  https://www.bayut.om/en/oman/properties-for-rent/page-13/
scarpping page:  https://www.bayut.om/en/

In [ ]:
df1 = pd.DataFrame(all_apt)
df1.to_csv("bayut.csv")